<a href="https://colab.research.google.com/github/NKalavros/Copehnagen-biohackathon/blob/main/Copy_of_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Linux stuff
!pip install seqvec

!wget https://rostlab.org/~deepppi/seqvec.zip
!unzip seqvec.zip

!wget https://rostlab.org/~deepppi/seqvec_checkpoint.tar.gz
!tar -zxf seqvec_checkpoint.tar.gz

!rm seqvec.zip seqvec_checkpoint.tar.gz

!wget https://www.dropbox.com/s/snl3v972umpy0k4/hackathon.csv?dl=0 -O hackathon.csv

!!pip install 'umap-learn==0.3.10'

     |████████████████████████████████| 133kB 4.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4
--2021-04-25 08:38:52--  https://rostlab.org/~deepppi/seqvec.zip
Resolving rostlab.org (rostlab.org)... 131.159.28.73
Connecting to rostlab.org (rostlab.org)|131.159.28.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347361261 (331M) [application/zip]
Saving to: ‘seqvec.zip’

seqvec.zip          100%[===================>] 331.27M  10.6MB/s    in 33s     

2021-04-25 08:39:26 (10.1 MB/s) - ‘seqvec.zip’ saved [347361261/347361261]

Archive:  seqvec.zip
replace uniref50_v2/weights.hdf5? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
--2021-04-25 08:43:30--  https://rostlab.org/~deepppi/seqvec_checkpoint.tar.gz
Resolving rostlab.org (rostlab.org)... 131.159.28.73
Connecting to rostlab

In [ ]:
#Imports
import torch
import csv

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_mutual_info_score as ads
from sklearn.metrics import normalized_mutual_info_score as nis
from sklearn.metrics import mutual_info_score as mis

#Imports
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable

import numpy as np
import math

#Utils
'''Some helper functions for PyTorch, including:
    - get_mean_and_std: calculate the mean and std value of dataset.
    - msr_init: net parameter initialization.
    - progress_bar: progress bar mimic xlua.progress.
'''
import os
import sys
import time
import math

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.data as data
from PIL import Image
from torch import FloatTensor, LongTensor
#Evaluation plots
import pandas as pd
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
import umap

#Ops
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.autograd import Variable
import torch.nn.functional as F
import math

import tensorflow as tf

In [ ]:
#Replacing this class with our own dataset class
class Dataset(data.Dataset):
    def __init__(self, data, labels, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        img, target = self.data[index], self.labels[index]
        # img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

class CDRH3MotifDataset(data.Dataset):
    """
    pyTorch Dataloader conform class,
    requires implementation of __len__ and __getitem__
    """

    def __init__(self, csv_path: str, device: str = 'cuda:0'):
        """
        Args:
            csv_path (string): Path to the csv file with CDRH3 motifs.
            device (string): device where data is kept in memory
        """
        with open(csv_path, 'r') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)
            classes = set()
            data = list()
            for line in reader:
                data.append((line[0], line[1:]))
                classes |= set(line[0])

        encoding = {letter: np.eye(len(classes), dtype='float32')[i]
                    for i, letter in enumerate(classes)}
        label_encoding = {letter: i for i, letter in enumerate(classes)}
        dataset = np.zeros((len(data), len(data[0][0]), len(classes)),
                           dtype='int')
        labels = np.zeros((len(data), len(data[0][0])), dtype='int64')
        eval_data = dict()
        for i, line in enumerate(data):
            for jdx, letter in enumerate(line[0]):
                dataset[i, jdx, :] = encoding[letter]
                labels[i, jdx] = label_encoding[letter]
                eval_data[i] = line[1]

        # transform numpy array to torch tensor
        # and copy it on the correct device
        self.dataset = torch.from_numpy(dataset).to(device)
        self.labels = torch.from_numpy(labels).to(device)
        self.eval_data = eval_data
        self.encoding = encoding
        self.label_encoding = label_encoding
        self.reverse_encoding = {repr(i): j for j, i in encoding.items()}

    def __len__(self) -> int:
        return len(self.dataset)

    """
    __getitem__
    Output:
        (OneHotEncoding of data, IntegerEncoding of data,
        list with epitope, structure and antigen)
    """
    def __getitem__(self, idx: int) -> (FloatTensor):
        return (self.dataset[idx, :, :],self.labels[idx])
    
def masking_noise(data, frac):
    """
    data: Tensor
    frac: fraction of unit to be masked out
    """
    data_noise = data.clone()
    rand = torch.rand(data.size())
    data_noise[rand<frac] = 0
    return data_noise


class MSELoss(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()

    def forward(self, input, target):
        return torch.mean(torch.sum((input-target)**2, 1))

class BCELoss(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()

    def forward(self, input, target):
        return -torch.mean(torch.sum(target*torch.log(torch.clamp(input, min=1e-10))+
            (1-target)*torch.log(torch.clamp(1-input, min=1e-10)), 1))

In [ ]:
#Simple DAE
class DenoisingAutoencoder(nn.Module):
    def __init__(self, in_features, out_features, activation="relu", 
        dropout=0.2, tied=False):
        super(self.__class__, self).__init__()
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if tied:
            self.deweight = self.weight.t()
        else:
            self.deweight = Parameter(torch.Tensor(in_features, out_features))
        self.bias = Parameter(torch.Tensor(out_features))
        self.vbias = Parameter(torch.Tensor(in_features))
        
        if activation=="relu":
            self.enc_act_func = nn.ReLU()
        elif activation=="sigmoid":
            self.enc_act_func = nn.Sigmoid()
        self.dropout = nn.Dropout(p=dropout)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        stdv = 1. / math.sqrt(self.deweight.size(1))
        self.deweight.data.uniform_(-stdv, stdv)
        self.vbias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        return self.dropout(self.enc_act_func(F.linear(x, self.weight, self.bias)))

    def encode(self, x, train=True):
        if train:
            self.dropout.train()
        else:
            self.dropout.eval()
        return self.dropout(self.enc_act_func(F.linear(x, self.weight, self.bias)))

    def encodeBatch(self, dataloader):
        use_cuda = torch.cuda.is_available()
        encoded = []
        for batch_idx, (inputs) in enumerate(dataloader):
            inputs = inputs.view(inputs.size(0), -1).float()
            if use_cuda:
                inputs = inputs.cuda()
            inputs = Variable(inputs)
            hidden = self.encode(inputs, train=False)
            encoded.append(hidden.data.cpu())

        encoded = torch.cat(encoded, dim=0)
        return encoded

    def decode(self, x, binary=False):
        if not binary:
            return F.linear(x, self.deweight, self.vbias)
        else:
            return F.sigmoid(F.linear(x, self.deweight, self.vbias))

    def fit(self, trainloader, lr=0.001, batch_size=235000, num_epochs=10, corrupt=0.3,
        loss_type="mse"):
        """
        data_x: FloatTensor
        valid_x: FloatTensor
        """
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        print("=====Denoising Autoencoding layer=======")
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lr)
        if loss_type=="mse":
            criterion = MSELoss()
        elif loss_type=="cross-entropy":
            criterion = BCELoss()

        # validate
        #total_loss = 0.0
        #total_num = 0
        #for batch_idx, (inputs, _) in enumerate(validloader):
        #    inputs = inputs.view(inputs.size(0), -1).float()
        #    if use_cuda:
        #        inputs = inputs.cuda()
        #    inputs = Variable(inputs)
        #    hidden = self.encode(inputs)
        #    if loss_type=="cross-entropy":
        #        outputs = self.decode(hidden, binary=True)
        #    else:
        #        outputs = self.decode(hidden)

        #    valid_recon_loss = criterion(outputs, inputs)
        #    total_loss += valid_recon_loss.data[0] * len(inputs)
        #    total_num += inputs.size()[0]

        #valid_loss = total_loss / total_num
        #print("#Epoch 0: Valid Reconstruct Loss: %.3f" % (valid_loss))

        for epoch in range(num_epochs):
            # train 1 epoch
            train_loss = 0.0
            for batch_idx, (inputs) in enumerate(trainloader):
                if self.activation != "tanh":
                    inputs = inputs.view(inputs.size(0), -1).float()
                inputs_corr = masking_noise(inputs, corrupt)
                if use_cuda:
                    inputs = inputs.cuda()
                    inputs_corr = inputs_corr.cuda()
                optimizer.zero_grad()
                inputs = Variable(inputs)
                inputs_corr = Variable(inputs_corr)

                hidden = self.encode(inputs_corr)
                if loss_type=="cross-entropy":
                    outputs = self.decode(hidden, binary=True)
                else:
                    outputs = self.decode(hidden)
                if self.activation != "tanh":
                    recon_loss = criterion(outputs, inputs)
                else:
                    recon_loss = criterion(outputs.reshape(inputs.shape),inputs)
                train_loss += recon_loss.data.item()*len(inputs)
                recon_loss.backward()
                optimizer.step()

            # validate
            #valid_loss = 0.0
            #for batch_idx, (inputs, _) in enumerate(validloader):
            #    inputs = inputs.view(inputs.size(0), -1).float()
            #    if use_cuda:
            #        inputs = inputs.cuda()
            #    inputs = Variable(inputs)
            #    hidden = self.encode(inputs, train=False)
            #    if loss_type=="cross-entropy":
            #        outputs = self.decode(hidden, binary=True)
            #    else:
            #        outputs = self.decode(hidden)
            #
            #    valid_recon_loss = criterion(outputs, inputs)
            #    valid_loss += valid_recon_loss.data[0] * len(inputs)

            print("#Epoch %3d: Reconstruct Loss: %.3f" % (
                epoch+1, train_loss / len(trainloader.dataset)))

In [ ]:
#Moving onto stacked DAE

def buildNetwork(layers, activation="relu", dropout=0,embedding = False):
    net = []
    print(layers)
    for i in range(1, len(layers)):
        if activation=="relu":
            net.append(nn.Linear(layers[i-1], layers[i]))
            net.append(nn.ReLU())
        elif activation=="tanh":
            new_input_dim = tuple([BATCH_SIZE] + layers[0])
            net.append(nn.GRU(input_size = layers[0][1],hidden_size = 32,
                              num_layers = layers[1],batch_first = True,dropout = dropout))
            break
        elif activation=="sigmoid":
            net.append(nn.Sigmoid())
        if activation != "tanh":
            if dropout > 0:
                net.append(nn.Dropout(dropout))
    return nn.Sequential(*net)

class StackedDAE(nn.Module):
    def __init__(self, input_dim=784, z_dim=10, binary=True,
        encodeLayer=[400], decodeLayer=[400], activation="relu", 
        dropout=0, tied=False,embedding = False):
        super(self.__class__, self).__init__()
        self.z_dim = z_dim
        self.layers = [input_dim] + encodeLayer + [z_dim]
        self.activation = activation
        self.dropout = dropout
        self.embedding = embedding
        self.encoder = buildNetwork([input_dim] + encodeLayer, activation=activation, dropout=dropout,
                                    embedding = self.embedding)
        if activation=="tanh":
            self.decoder = buildNetwork([[z_dim,z_dim]] + decodeLayer, activation=activation, dropout=dropout,
                                        embedding = self.embedding)
        else:
            self.decoder = buildNetwork([z_dim] + [decodeLayer], activation=activation, dropout=dropout,
                                        embedding = self.embedding)
        if self.activation != "tanh":
            self._enc_mu = nn.Linear(11*20*64, z_dim)
        
            self._dec = nn.Linear(decodeLayer[-1], input_dim)
            self._dec_act = None
        else:
            self._enc_mu = nn.Linear(32*11,z_dim)
            print(input_dim[0], input_dim[1],input_dim[0]*input_dim[1])
            self._dec = nn.Linear(32,input_dim[0]*input_dim[1])
            self._dec_act = "None"
        if binary:
            self._dec_act = nn.Sigmoid()

    def decode(self, z):
        if self.activation != "tanh":
            h = self.decoder(z)
            x = self._dec(h)
        else:
            z_new = z.unsqueeze(1)
            #print("decoder z shape",z_new.shape)
            h, h_n = self.decoder(z_new)
            h_new = h.reshape((h.shape[0],-1))
            #print("hnew shape",h_new.shape)
            x = self._dec(h_new)
            #print("x shape",x.shape)
        if self._dec_act is not None:
            x = self._dec_act(x)
        return x

    def loss_function(self, recon_x, x):
        loss = -torch.mean(torch.sum(x*torch.log(torch.clamp(recon_x, min=1e-10))+
            (1-x)*torch.log(torch.clamp(1-recon_x, min=1e-10)), 1))

        return loss

    def forward(self, x):
        if(self.activation != "tanh"):
            h = self.encoder(x)
            z = self._enc_mu(h)
        else:
            h,h_n = self.encoder(x)
            h_new = h.reshape((h.shape[0],-1))
            #print("hnew shape",h_new.shape)
            z = self._enc_mu(h_new)
            #print("z shape",z.shape)
        return z, self.decode(z)

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict) 
        self.load_state_dict(model_dict)

    def pretrain(self, trainloader, validloader, lr=0.001, batch_size=128, num_epochs=10, corrupt=0.3, loss_type="cross-entropy"):
        trloader = trainloader
        valoader = validloader
        daeLayers = []
        for l in range(1, len(self.layers)):
            infeatures = self.layers[l-1]
            outfeatures = self.layers[l]
            dae = DenoisingAutoencoder(infeatures, outfeatures, activation=self.activation, dropout=self.dropout)
            if l==1:
                dae.fit(trloader, valoader, lr=lr, batch_size=batch_size, num_epochs=num_epochs, corrupt=corrupt, loss_type=loss_type)
            else:
                if self.activation=="sigmoid":
                    dae.fit(trloader, valoader, lr=lr, batch_size=batch_size, num_epochs=num_epochs, corrupt=corrupt, loss_type="cross-entropy")
                else:
                    dae.fit(trloader, valoader, lr=lr, batch_size=batch_size, num_epochs=num_epochs, corrupt=corrupt, loss_type="mse")
            data_x = dae.encodeBatch(trloader)
            valid_x = dae.encodeBatch(valoader)
            trainset = Dataset(data_x, data_x)
            trloader = torch.utils.data.DataLoader(
                trainset, batch_size=batch_size, shuffle=True, num_workers=2)
            validset = Dataset(valid_x, valid_x)
            valoader = torch.utils.data.DataLoader(
                validset, batch_size=1000, shuffle=False, num_workers=2)
            daeLayers.append(dae)

        self.copyParam(daeLayers)

    def copyParam(self, daeLayers):
        if self.dropout==0:
            every = 2
        else:
            every = 3
        # input layer
        # copy encoder weight
        self.encoder[0].weight.data.copy_(daeLayers[0].weight.data)
        self.encoder[0].bias.data.copy_(daeLayers[0].bias.data)
        self._dec.weight.data.copy_(daeLayers[0].deweight.data)
        self._dec.bias.data.copy_(daeLayers[0].vbias.data)

        for l in range(1, len(self.layers)-2):
            # copy encoder weight
            self.encoder[l*every].weight.data.copy_(daeLayers[l].weight.data)
            self.encoder[l*every].bias.data.copy_(daeLayers[l].bias.data)

            # copy decoder weight
            self.decoder[-(l-1)*every-2].weight.data.copy_(daeLayers[l].deweight.data)
            self.decoder[-(l-1)*every-2].bias.data.copy_(daeLayers[l].vbias.data)

        # z layer
        self._enc_mu.weight.data.copy_(daeLayers[-1].weight.data)
        self._enc_mu.bias.data.copy_(daeLayers[-1].bias.data)
        self.decoder[0].weight.data.copy_(daeLayers[-1].deweight.data)
        self.decoder[0].bias.data.copy_(daeLayers[-1].vbias.data)

    def fit(self, trainloader, lr=0.001, num_epochs=10, corrupt=0.3,
        loss_type="mse"):
        """
        data_x: FloatTensor
        valid_x: FloatTensor
        """
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        print("=====Stacked Denoising Autoencoding layer=======")
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lr)
        if loss_type=="mse":
            criterion = MSELoss()
        elif loss_type=="cross-entropy":
            criterion = BCELoss()

        for epoch in range(num_epochs):
            # train 1 epoch
            train_loss = 0.0
            for batch_idx, (inputs) in enumerate(trainloader):
                if self.activation == "relu":
                    inputs = inputs.view(inputs.size(0), -1).float()
                inputs_corr = masking_noise(inputs, corrupt)
                if use_cuda:
                    inputs = inputs.cuda()
                    inputs_corr = inputs_corr.cuda()
                optimizer.zero_grad()
                inputs = Variable(inputs)
                inputs_corr = Variable(inputs_corr)

                z, outputs = self.forward(inputs_corr)
                #print(inputs_corr.shape)
                #print(z.shape)
                #print(outputs.shape)
                if self.activation != "tanh":
                    recon_loss = criterion(outputs, inputs)
                else:
                    recon_loss = criterion(outputs.reshape(inputs.shape),inputs)
                train_loss += recon_loss.data.item()*len(inputs)
                recon_loss.backward()
                optimizer.step()

            print("#Epoch %3d: Reconstruct Loss: %.3f" % (
                epoch+1, train_loss / len(trainloader.dataset)))

In [ ]:
def do_eval(z,do_tsne=False, do_umap = True):
    if "tensor" in str(type(z)):
        latent_repr = z.detach().cpu().numpy()
    else:
        latent_repr = z
    if latent_repr.shape[1] > 50:
        print("Big latent representation, reducing with PCA")
        pca = PCA(n_components=10)
        x_50 = pca.fit_transform(latent_repr)
        latent_repr2 = x_50
        print("Smaller latent representation, doing only PCs 1 and 2")
        pca = PCA(n_components=2)
        x = pca.fit_transform(latent_repr)
    else:
        print("Smaller latent representation, doing only PCs 1 and 2")
        pca = PCA(n_components=2)
        x = pca.fit_transform(latent_repr)
        latent_repr2 = latent_repr
    
    
    with open(DATA_PATH, 'r') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)
            classes = set()
            data = list()
            for line in reader:
                data.append((line[0], line[1:]))
                classes |= set(line[0])
    labels = [x[1][2] for x in data]
    ag = labels
    ag_set = set()
    for element in ag:
        ag_set |= {element}
    ag_map = {name: i for i, name in enumerate(ag_set)}
    ag_color = [ag_map[s] for s in ag]
    
    #tSNE
    if do_tsne:
        print("Doing tSNE")
        tsne = TSNE(n_components=2, verbose=1, random_state=123,n_jobs = 10)  
        z = tsne.fit_transform(latent_repr2)
        df = pd.DataFrame()
        df["y"] = ag
        df["comp-1"] = z[:, 0]
        df["comp-2"] = z[:, 1]
        sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(), legend='full',
                         palette=sns.color_palette("hls", 10),
                         data=df).set(title="Antigen T-SNE projection")
        plt.savefig("tSNE_output.png")
    
    # UMAP
    if do_umap:
        print("Doing UMAP")
        ag_umap = umap.UMAP()
        embedding = pd.DataFrame(ag_umap.fit_transform(latent_repr2),
                                 columns=['UMAP1', 'UMAP2'])
        sns_plot = sns.scatterplot(x='UMAP1', y='UMAP2', data=embedding, hue=np.array(ag),
                                   alpha=.5, linewidth=0, s=1)
        sns_plot.legend(bbox_to_anchor=(1.03, 1), fontsize=12)
        sns_plot.figure.savefig('UMAP_output.png', bbox_inches='tight', dpi=500)
    
    plt.figure()
    plt.scatter(x[:, 0], x[:, 1], c=np.array(ag_color),
                cmap=plt.get_cmap('tab10'), alpha=0.5)
    plt.savefig('tab10.png')
    
    print("Doing k-means")
    # clustering
    kmeans = KMeans(n_clusters=10).fit(latent_repr)
    print("Doing GMM")
    gm = GaussianMixture(n_components=10, random_state=0).fit(latent_repr)
    #Cluster purity
        # computing cluster purity
    purity = [0.0] * 10
    occ_epis = {new_list: [0] * 10 for new_list in range(10)}
    seq_clusters = [0] * 10

    for i, j in zip(np.array(ag_color), kmeans.labels_):
        seq_clusters[j] += 1
        tmp = occ_epis.get(j)
        tmp[i] += 1
        occ_epis.update({j: tmp})

    for j in range(len(seq_clusters)):
        purity[j] = max(occ_epis[j]) / seq_clusters[j]
    # eval clustering
    print("Kmeans")
    print("Adjusted MI:", ads(np.array(ag_color), kmeans.labels_))
    print("Normalized MI:",nis(np.array(ag_color), kmeans.labels_))
    print("Purity:", purity)
    print("GMM")
    print("Adjusted MI:", ads(np.array(ag_color), gm.predict(latent_repr)))
    print("Normalized MI:",nis(np.array(ag_color),  gm.predict(latent_repr)))
    print("Purity:", purity)

In [ ]:
#Training variables
DATA_PATH = 'hackathon.csv'
EPOCHS = 5
BATCH_SIZE = 5000
LATENT_N = 32 #Bigg.
DEVICE = 'cuda:0'  # change to 'cpu' if training on cpu

from torch.utils.data import Dataset, DataLoader
#Load data
dataset = CDRH3MotifDataset(DATA_PATH, device=DEVICE)


In [ ]:
with open(DATA_PATH, 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader)
    classes = set()
    data = list()
    for line in reader:
        data.append((line[0], line[1:]))
        classes |= set(line[0])

In [ ]:
peptides = [x[0] for x in data]

In [ ]:
from pathlib import Path

os.getcwd()
model_dir = Path(os.path.join(os.getcwd(),"uniref50_v2"))
weights = model_dir / 'weights.hdf5'
options = model_dir / 'options.json'

In [ ]:
import numpy as np
import math
from allennlp.commands.elmo import ElmoEmbedder

embedder = ElmoEmbedder(options,weights, cuda_device=0) # cuda_device=-1 for CPU

In [ ]:
peptideslist = [list(x) for x in peptides]

In [ ]:
peptideslist.sort(key=len)

In [ ]:
embedding = embedder.embed_sentences(peptideslist) # returns: List-of-Lists with shape [3,L,1024]

In [ ]:
my_tensor = torch.ones((len(dataset),11,1024), dtype=torch.float, device=DEVICE, requires_grad=False)
for i in enumerate(embedding):
    temp_tensor = i[1].sum(0)
    my_tensor[i[0],:,:] = torch.Tensor(temp_tensor).to(DEVICE)

In [ ]:
data = DataLoader(my_tensor, batch_size=BATCH_SIZE, shuffle=True, num_workers=0,
                  drop_last=True)

In [ ]:
for i in enumerate(data):
    temp_tensor = i[1]
    print(temp_tensor.shape)
    feature_shape = temp_tensor.shape[1]*temp_tensor.shape[2]
    break

In [ ]:
for i in [(1200,1200),(600,600),(500,500),(400,400),(300,300),(200,200),(100,100)]:
    break
    SDAE = StackedDAE(feature_shape,LATENT_N,False,[i[0]],[i[1]],dropout=0.0)
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.1)

    data = DataLoader(my_tensor, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
feature_shape = [11,1024]

In [ ]:
torch.cuda.empty_cache()
#LSTM autoencoder
for i in [(300,300),(200,200),(100,100)]:

    SDAE = StackedDAE(feature_shape,LATENT_N,False,[i[0]//100],[i[1]//100],dropout=0.5,
                      activation="tanh")
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.4)

    data = DataLoader(my_tensor, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
torch.cuda.empty_cache()
#LSTM autoencoder
for i in [(300,300),(200,200),(100,100)]:

    SDAE = StackedDAE(feature_shape,LATENT_N,False,[i[0]//100],[i[1]//100],dropout=0.5,
                      activation="tanh")
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.4)

    data = DataLoader(my_tensor, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
#Moving onto stacked DAE

def buildNetwork2(layers, activation="relu", dropout=0,embedding = False):
    net = []
    print(layers)
    net.append(nn.Embedding(20,64))
    for i in range(1, len(layers)):
        if activation=="relu":
            net.append(nn.Linear(64, 64))
            net.append(nn.ReLU())
        elif activation=="tanh":
            new_input_dim = tuple([BATCH_SIZE] + layers[0])
            net.append(nn.GRU(input_size = 64,hidden_size = 32,
                              num_layers = layers[1],batch_first = True,dropout = dropout))
            break
        elif activation=="sigmoid":
            net.append(nn.Sigmoid())
        if activation != "tanh":
            if dropout > 0:
                net.append(nn.Dropout(dropout))
    return nn.Sequential(*net)

def buildNetwork3(layers, activation="relu", dropout=0,embedding = True):
    net = []
    print(layers)
    for i in range(1, len(layers)):
        if activation=="relu":
            net.append(nn.Linear(layers[i-1], layers[i][0]))
            net.append(nn.ReLU())
        elif activation=="tanh":
            new_input_dim = tuple([BATCH_SIZE] + layers[0])
            net.append(nn.GRU(input_size = 64,hidden_size = 32,
                              num_layers = layers[1],batch_first = True,dropout = dropout))
            break
        elif activation=="sigmoid":
            net.append(nn.Sigmoid())
        if activation != "tanh":
            if dropout > 0:
                net.append(nn.Dropout(dropout))
    return nn.Sequential(*net)

class StackedDAE2(nn.Module):
    def __init__(self, input_dim=784, z_dim=10, binary=True,
        encodeLayer=[400], decodeLayer=[400], activation="relu", 
        dropout=0, tied=False,embedding = False):
        super(self.__class__, self).__init__()
        self.z_dim = z_dim
        self.layers = [input_dim] + encodeLayer + [z_dim]
        self.activation = activation
        self.dropout = dropout
        self.embedding = embedding
        self.encoder = buildNetwork2([input_dim] + encodeLayer, activation=activation, dropout=dropout,
                                    embedding = self.embedding)
        if activation=="tanh":
            self.decoder = buildNetwork2([[z_dim,z_dim]] + decodeLayer, activation=activation, dropout=dropout,
                                        embedding = self.embedding)
        else:
            self.decoder = buildNetwork3([z_dim] + [decodeLayer], activation=activation, dropout=dropout,
                                         embedding = self.embedding)
        if self.activation != "tanh":
            self._enc_mu = nn.Linear(11*20*64, z_dim)
            print(decodeLayer[-1])
            self._dec = nn.Linear(decodeLayer[-1], input_dim[0]*input_dim[1])
            self._dec_act = None
        else:
            self._enc_mu = nn.Linear(32*encodeLayer[-1],z_dim)
            print(input_dim[0], input_dim[1],input_dim[0]*input_dim[1])
            self._dec = nn.Linear(32,input_dim[0]*input_dim[1])
            self._dec_act = None
        if binary:
            self._dec_act = nn.Sigmoid()

    def decode(self, z):
        if self.activation != "tanh":
            h = self.decoder(z)
            x = self._dec(h)
        else:
            z_new = z.unsqueeze(1)
            #print("decoder z shape",z_new.shape)
            h, h_n = self.decoder(z_new)
            h_new = h.reshape((h.shape[0],-1))
            #print("hnew shape",h_new.shape)
            x = self._dec(h_new)
            #print("x shape",x.shape)
        if self._dec_act is not None:
            x = self._dec_act(x)
        return x

    def loss_function(self, recon_x, x):
        loss = -torch.mean(torch.sum(x*torch.log(torch.clamp(recon_x, min=1e-10))+
            (1-x)*torch.log(torch.clamp(1-recon_x, min=1e-10)), 1))

        return loss

    def forward(self, x):
        if(self.activation != "tanh"):
            #print("x shape",x.shape)
            h = self.encoder(x)
            #print("h shape",h.shape)
            h_new = h.reshape(h.shape[0],-1)
            #print("h new hspae",h_new.shape)
            z = self._enc_mu(h_new)
        else:
            h,h_n = self.encoder(x)
            h_new = h.reshape((h.shape[0],-1))
            #print("hnew shape",h_new.shape)
            z = self._enc_mu(h_new)
            #print("z shape",z.shape)
        return z, self.decode(z)

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict) 
        self.load_state_dict(model_dict)

    def pretrain(self, trainloader, validloader, lr=0.001, batch_size=128, num_epochs=10, corrupt=0.3, loss_type="cross-entropy"):
        trloader = trainloader
        valoader = validloader
        daeLayers = []
        for l in range(1, len(self.layers)):
            infeatures = self.layers[l-1]
            outfeatures = self.layers[l]
            dae = DenoisingAutoencoder(infeatures, outfeatures, activation=self.activation, dropout=self.dropout)
            if l==1:
                dae.fit(trloader, valoader, lr=lr, batch_size=batch_size, num_epochs=num_epochs, corrupt=corrupt, loss_type=loss_type)
            else:
                if self.activation=="sigmoid":
                    dae.fit(trloader, valoader, lr=lr, batch_size=batch_size, num_epochs=num_epochs, corrupt=corrupt, loss_type="cross-entropy")
                else:
                    dae.fit(trloader, valoader, lr=lr, batch_size=batch_size, num_epochs=num_epochs, corrupt=corrupt, loss_type="mse")
            data_x = dae.encodeBatch(trloader)
            valid_x = dae.encodeBatch(valoader)
            trainset = Dataset(data_x, data_x)
            trloader = torch.utils.data.DataLoader(
                trainset, batch_size=batch_size, shuffle=True, num_workers=2)
            validset = Dataset(valid_x, valid_x)
            valoader = torch.utils.data.DataLoader(
                validset, batch_size=1000, shuffle=False, num_workers=2)
            daeLayers.append(dae)

        self.copyParam(daeLayers)

    def copyParam(self, daeLayers):
        if self.dropout==0:
            every = 2
        else:
            every = 3
        # input layer
        # copy encoder weight
        self.encoder[0].weight.data.copy_(daeLayers[0].weight.data)
        self.encoder[0].bias.data.copy_(daeLayers[0].bias.data)
        self._dec.weight.data.copy_(daeLayers[0].deweight.data)
        self._dec.bias.data.copy_(daeLayers[0].vbias.data)

        for l in range(1, len(self.layers)-2):
            # copy encoder weight
            self.encoder[l*every].weight.data.copy_(daeLayers[l].weight.data)
            self.encoder[l*every].bias.data.copy_(daeLayers[l].bias.data)

            # copy decoder weight
            self.decoder[-(l-1)*every-2].weight.data.copy_(daeLayers[l].deweight.data)
            self.decoder[-(l-1)*every-2].bias.data.copy_(daeLayers[l].vbias.data)

        # z layer
        self._enc_mu.weight.data.copy_(daeLayers[-1].weight.data)
        self._enc_mu.bias.data.copy_(daeLayers[-1].bias.data)
        self.decoder[0].weight.data.copy_(daeLayers[-1].deweight.data)
        self.decoder[0].bias.data.copy_(daeLayers[-1].vbias.data)

    def fit(self, trainloader, lr=0.001, num_epochs=10, corrupt=0.3,
        loss_type="mse"):
        """
        data_x: FloatTensor
        valid_x: FloatTensor
        """
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        print("=====Stacked Denoising Autoencoding layer=======")
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lr)
        if loss_type=="mse":
            criterion = MSELoss()
        elif loss_type=="cross-entropy":
            criterion = BCELoss()

        for epoch in range(num_epochs):
            # train 1 epoch
            train_loss = 0.0
            for batch_idx, (inputs) in enumerate(trainloader):
                if self.activation == "relu":
                    pass
                    #inputs = inputs.view(inputs.size(0), -1).float()
                inputs_corr = masking_noise(inputs, corrupt)
                if use_cuda:
                    inputs = inputs.cuda()
                    inputs_corr = inputs_corr.cuda()
                optimizer.zero_grad()
                inputs = Variable(inputs)
                inputs_corr = Variable(inputs_corr)

                z, outputs = self.forward(inputs_corr)
                #print(inputs_corr.shape)
                #print(z.shape)
                #print(outputs.shape)
                outputs = outputs.reshape(inputs.shape)
                if self.activation != "tanh":
                    recon_loss = criterion(outputs, inputs)
                else:
                    recon_loss = criterion(outputs.reshape(inputs.shape),inputs)
                train_loss += recon_loss.data.item()*len(inputs)
                recon_loss.backward()
                optimizer.step()

            print("#Epoch %3d: Reconstruct Loss: %.3f" % (
                epoch+1, train_loss / len(trainloader.dataset)))

In [ ]:
data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0,
                  drop_last=True)

In [ ]:
for i in data:
    print(i)
    break

In [ ]:
for i in [(1200,1200),(600,600),(500,500),(400,400),(300,300),(200,200),(100,100)]:
    SDAE = StackedDAE2([11,20],LATENT_N,True,[i[0]],[i[1]],dropout=0.0)
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.1)

    data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
for i in [(1200,1200),(600,600),(500,500),(400,400),(300,300),(200,200),(100,100)]:
    SDAE = StackedDAE2([11,20],LATENT_N,True,[i[0]],[i[1]],dropout=0.4)
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.5)

    data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
LATENT_N = 32
for i in [(1200,1200),(600,600),(500,500),(400,400),(300,300),(200,200),(100,100)]:
    SDAE = StackedDAE2([11,20],LATENT_N,True,[i[0]],[i[1]],dropout=0.4)
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.5)

    data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
LATENT_N = 16
for i in [(1200,1200),(600,600),(500,500),(400,400),(300,300),(200,200),(100,100)]:
    SDAE = StackedDAE2([11,20],LATENT_N,True,[i[0]],[i[1]],dropout=0.4)
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.5)

    data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
LATENT_N = 64
for i in [(1200,1200),(600,600),(500,500),(400,400),(300,300),(200,200),(100,100)]:
    SDAE = StackedDAE2([11,20],LATENT_N,True,[i[0]],[i[1]],dropout=0.3)
    SDAE.fit(data,num_epochs=EPOCHS,corrupt=0.3)

    data = DataLoader(dataset, batch_size=BATCH_SbIZE, shuffle=False, num_workers=0,
                    drop_last = False)

    all_z = np.zeros((len(dataset),LATENT_N),np.float32)
    with torch.no_grad():
        for batch_idx, (inputs) in enumerate(data):
            #inputs = inputs.view(inputs.size(0), -1).float()
            z, outputs = SDAE.forward(inputs)
            if z.shape[0] == BATCH_SIZE:
                all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z.cpu().numpy()
            else:
                all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z.cpu().numpy()

    do_eval(all_z,do_umap = False)

In [ ]:
!pip install gpflow

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

import numpy as np
import math
from sklearn.mixture import GaussianMixture

def cluster_acc(Y_pred, Y):
  from sklearn.utils.linear_assignment_ import linear_assignment
  assert Y_pred.size == Y.size
  D = max(Y_pred.max(), Y.max())+1
  D = int(D)
  w = np.zeros((D,D), dtype=np.float64)
  for i in range(Y_pred.size):
    Y = [int(x) for x in Y]
    w[Y_pred[i], Y[i]] += 1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w

def buildNetwork(layers, activation="relu", dropout=0):
    net = []
    for i in range(1, len(layers)):
        net.append(nn.Linear(layers[i-1], layers[i]))
        if activation=="relu":
            net.append(nn.ReLU())
        elif activation=="sigmoid":
            net.append(nn.Sigmoid())
        if dropout > 0:
            net.append(nn.Dropout(dropout))
    return nn.Sequential(*net)

def adjust_learning_rate(init_lr, optimizer, epoch):
    lr = max(init_lr * (0.9 ** (epoch//10)), 0.0002)
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
    return lr

log2pi = math.log(2*math.pi)
def log_likelihood_samples_unit_gaussian(samples):
    return -0.5*log2pi*samples.size()[1] - torch.sum(0.5*(samples)**2, 1)

def log_likelihood_samplesImean_sigma(samples, mu, logvar):
    return -0.5*log2pi*samples.size()[1] - torch.sum(0.5*(samples-mu)**2/torch.exp(logvar) + 0.5*logvar, 1)

class VaDE(nn.Module):
    def __init__(self, input_dim=784, z_dim=10, n_centroids=10, binary=True,
        encodeLayer=[500,500,2000], decodeLayer=[2000,500,500]):
        super(self.__class__, self).__init__()
        self.z_dim = z_dim
        self.n_centroids = n_centroids
        self.encoder = buildNetwork([input_dim] + encodeLayer)
        self.decoder = buildNetwork([z_dim] + decodeLayer)
        self._enc_mu = nn.Linear(encodeLayer[-1], z_dim)
        self._enc_log_sigma = nn.Linear(encodeLayer[-1], z_dim)
        self._dec = nn.Linear(decodeLayer[-1], input_dim)
        self._dec_act = None
        if binary:
            self._dec_act = nn.Sigmoid()

        self.create_gmmparam(n_centroids, z_dim)

    def create_gmmparam(self, n_centroids, z_dim):
        self.theta_p = nn.Parameter(torch.ones(n_centroids)/n_centroids)
        self.u_p = nn.Parameter(torch.zeros(z_dim, n_centroids))
        self.lambda_p = nn.Parameter(torch.ones(z_dim, n_centroids))

    def initialize_gmm(self, dataloader):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()

        self.eval()
        data = []
        for batch_idx, (inputs,labels) in enumerate(dataloader):
            inputs = inputs.view(inputs.size(0), -1).float()
            if use_cuda:
                inputs = inputs.cuda()
            inputs = Variable(inputs)
            z, outputs, mu, logvar = self.forward(inputs)
            data.append(z.data.cpu().numpy())
        data = np.concatenate(data)
        gmm = GaussianMixture(n_components=self.n_centroids,covariance_type='diag')
        gmm.fit(data)
        self.u_p.data.copy_(torch.from_numpy(gmm.means_.T.astype(np.float32)))
        self.lambda_p.data.copy_(torch.from_numpy(gmm.covariances_.T.astype(np.float32)))

    def reparameterize(self, mu, logvar):
        if self.training:
          std = logvar.mul(0.5).exp_()
          eps = Variable(std.data.new(std.size()).normal_())
          # num = np.array([[ 1.096506  ,  0.3686553 , -0.43172026,  1.27677995,  1.26733758,
          #       1.30626082,  0.14179629,  0.58619505, -0.76423112,  2.67965817]], dtype=np.float32)
          # num = np.repeat(num, mu.size()[0], axis=0)
          # eps = Variable(torch.from_numpy(num))
          return eps.mul(std).add_(mu)
        else:
          return mu

    def decode(self, z):
        h = self.decoder(z)
        x = self._dec(h)
        if self._dec_act is not None:
            x = self._dec_act(x)
        return x

    def get_gamma(self, z, z_mean, z_log_var):
        Z = z.unsqueeze(2).expand(z.size()[0], z.size()[1], self.n_centroids) # NxDxK
        z_mean_t = z_mean.unsqueeze(2).expand(z_mean.size()[0], z_mean.size()[1], self.n_centroids)
        z_log_var_t = z_log_var.unsqueeze(2).expand(z_log_var.size()[0], z_log_var.size()[1], self.n_centroids)
        u_tensor3 = self.u_p.unsqueeze(0).expand(z.size()[0], self.u_p.size()[0], self.u_p.size()[1]) # NxDxK
        lambda_tensor3 = self.lambda_p.unsqueeze(0).expand(z.size()[0], self.lambda_p.size()[0], self.lambda_p.size()[1])
        theta_tensor2 = self.theta_p.unsqueeze(0).expand(z.size()[0], self.n_centroids) # NxK

        p_c_z = torch.exp(torch.log(theta_tensor2) - torch.sum(0.5*torch.log(2*math.pi*lambda_tensor3)+\
            (Z-u_tensor3)**2/(2*lambda_tensor3), dim=1)) + 1e-10 # NxK
        gamma = p_c_z / torch.sum(p_c_z, dim=1, keepdim=True)

        return gamma

    def loss_function(self, recon_x, x, z, z_mean, z_log_var):
        Z = z.unsqueeze(2).expand(z.size()[0], z.size()[1], self.n_centroids) # NxDxK
        z_mean_t = z_mean.unsqueeze(2).expand(z_mean.size()[0], z_mean.size()[1], self.n_centroids)
        z_log_var_t = z_log_var.unsqueeze(2).expand(z_log_var.size()[0], z_log_var.size()[1], self.n_centroids)
        u_tensor3 = self.u_p.unsqueeze(0).expand(z.size()[0], self.u_p.size()[0], self.u_p.size()[1]) # NxDxK
        lambda_tensor3 = self.lambda_p.unsqueeze(0).expand(z.size()[0], self.lambda_p.size()[0], self.lambda_p.size()[1])
        theta_tensor2 = self.theta_p.unsqueeze(0).expand(z.size()[0], self.n_centroids) # NxK
        
        p_c_z = torch.exp(torch.log(theta_tensor2) - torch.sum(0.5*torch.log(2*math.pi*lambda_tensor3)+\
            (Z-u_tensor3)**2/(2*lambda_tensor3), dim=1)) + 1e-10 # NxK
        gamma = p_c_z / torch.sum(p_c_z, dim=1, keepdim=True) # NxK
        
        BCE = -torch.sum(x*torch.log(torch.clamp(recon_x, min=1e-10))+
            (1-x)*torch.log(torch.clamp(1-recon_x, min=1e-10)), 1)
        logpzc = torch.sum(0.5*gamma*torch.sum(math.log(2*math.pi)+torch.log(lambda_tensor3)+\
            torch.exp(z_log_var_t)/lambda_tensor3 + (z_mean_t-u_tensor3)**2/lambda_tensor3, dim=1), dim=1)
        qentropy = -0.5*torch.sum(1+z_log_var+math.log(2*math.pi), 1)
        logpc = -torch.sum(torch.log(theta_tensor2)*gamma, 1)
        logqcx = torch.sum(torch.log(gamma)*gamma, 1)

        # Normalise by same number of elements as in reconstruction
        loss = torch.mean(BCE + logpzc + qentropy + logpc + logqcx)

        return loss

    #===============================================================
    # below is defined according to the released code by the authors
    # However, they are incorrect in several places
    #===============================================================

    # def get_gamma(self, z, z_mean, z_log_var):
    #     Z = z.unsqueeze(2).expand(z.size()[0], z.size()[1], self.n_centroids) # NxDxK
    #     z_mean_t = z_mean.unsqueeze(2).expand(z_mean.size()[0], z_mean.size()[1], self.n_centroids)
    #     z_log_var_t = z_log_var.unsqueeze(2).expand(z_log_var.size()[0], z_log_var.size()[1], self.n_centroids)
    #     u_tensor3 = self.u_p.unsqueeze(0).expand(z.size()[0], self.u_p.size()[0], self.u_p.size()[1]) # NxDxK
    #     lambda_tensor3 = self.lambda_p.unsqueeze(0).expand(z.size()[0], self.lambda_p.size()[0], self.lambda_p.size()[1])
    #     theta_tensor3 = self.theta_p.unsqueeze(0).unsqueeze(1).expand(z.size()[0], z.size()[1], self.n_centroids) # NxDxK

    #     p_c_z = torch.exp(torch.sum(torch.log(theta_tensor3) - 0.5*torch.log(2*math.pi*lambda_tensor3)-\
    #         (Z-u_tensor3)**2/(2*lambda_tensor3), dim=1)) + 1e-10 # NxK
    #     gamma = p_c_z / torch.sum(p_c_z, dim=1, keepdim=True) # NxK

    #     return gamma

    # def loss_function(self, recon_x, x, z, z_mean, z_log_var):
    #     Z = z.unsqueeze(2).expand(z.size()[0], z.size()[1], self.n_centroids) # NxDxK
    #     z_mean_t = z_mean.unsqueeze(2).expand(z_mean.size()[0], z_mean.size()[1], self.n_centroids)
    #     z_log_var_t = z_log_var.unsqueeze(2).expand(z_log_var.size()[0], z_log_var.size()[1], self.n_centroids)
    #     u_tensor3 = self.u_p.unsqueeze(0).expand(z.size()[0], self.u_p.size()[0], self.u_p.size()[1]) # NxDxK
    #     lambda_tensor3 = self.lambda_p.unsqueeze(0).expand(z.size()[0], self.lambda_p.size()[0], self.lambda_p.size()[1])
    #     theta_tensor3 = self.theta_p.unsqueeze(0).unsqueeze(1).expand(z.size()[0], z.size()[1], self.n_centroids) # NxDxK

    #     p_c_z = torch.exp(torch.sum(torch.log(theta_tensor3) - 0.5*torch.log(2*math.pi*lambda_tensor3)-\
    #         (Z-u_tensor3)**2/(2*lambda_tensor3), dim=1)) + 1e-10 # NxK
    #     gamma = p_c_z / torch.sum(p_c_z, dim=1, keepdim=True) # NxK
    #     gamma_t = gamma.unsqueeze(1).expand(gamma.size(0), self.z_dim, gamma.size(1)) #
        
    #     BCE = -torch.sum(x*torch.log(torch.clamp(recon_x, min=1e-10))+
    #         (1-x)*torch.log(torch.clamp(1-recon_x, min=1e-10)), 1)
    #     logpzc = torch.sum(torch.sum(0.5*gamma_t*(self.z_dim*math.log(2*math.pi)+torch.log(lambda_tensor3)+\
    #         torch.exp(z_log_var_t)/lambda_tensor3 + (z_mean_t-u_tensor3)**2/lambda_tensor3), dim=1), dim=1)
    #     qentropy = -0.5*torch.sum(1+z_log_var+math.log(2*math.pi), 1)
    #     logpc = -torch.sum(torch.log(self.theta_p.unsqueeze(0).expand(z.size()[0], self.n_centroids))*gamma, 1)
    #     logqcx = torch.sum(torch.log(gamma)*gamma, 1)

    #     loss = torch.mean(BCE + logpzc + qentropy + logpc + logqcx)

    #     # return torch.mean(qentropy)
    #     return loss

    def forward(self, x):
        h = self.encoder(x)
        mu = self._enc_mu(h)
        logvar = self._enc_log_sigma(h)
        z = self.reparameterize(mu, logvar)
        return z, self.decode(z), mu, logvar

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict) 
        self.load_state_dict(model_dict)

    def fit(self, trainloader, validloader, lr=0.001, batch_size=128, num_epochs=10, 
        visualize=False, anneal=False):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()

        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lr)

        # validate
        self.eval()
        valid_loss = 0.0
        for batch_idx, (inputs,labels) in enumerate(validloader):
            inputs = inputs.view(inputs.size(0), -1).float()
            if use_cuda:
                inputs = inputs.cuda()
            inputs = Variable(inputs)
            z, outputs, mu, logvar = self.forward(inputs)

            loss = self.loss_function(outputs, inputs, z, mu, logvar)
            valid_loss += loss.data.item()*len(inputs)
            # total_loss += valid_recon_loss.data[0] * inputs.size()[0]
            # total_num += inputs.size()[0]

        # valid_loss = total_loss / total_num
        print("#Epoch -1: Valid Loss: %.5f" % (valid_loss / len(validloader.dataset)))

        for epoch in range(num_epochs):
            # train 1 epoch
            self.train()
            if anneal:
                epoch_lr = adjust_learning_rate(lr, optimizer, epoch)
            train_loss = 0
            for batch_idx, (inputs) in enumerate(trainloader):
                #print("Train batch idx", batch_idx)
                inputs = inputs.view(inputs.size(0), -1).float()
                if use_cuda:
                    inputs = inputs.cuda()
                optimizer.zero_grad()
                inputs = Variable(inputs)
                
                z, outputs, mu, logvar = self.forward(inputs)
                loss = self.loss_function(outputs, inputs, z, mu, logvar)
                train_loss += loss.data.item()*len(inputs)
                loss.backward()
                optimizer.step()
                # print("    #Iter %3d: Reconstruct Loss: %.3f" % (
                #     batch_idx, recon_loss.data[0]))

            # validate
            self.eval()
            valid_loss = 0.0
            Y = []
            Y_pred = []
            for batch_idx, (inputs, labels) in enumerate(validloader):
                #print("Val batch_idx",batch_idx)
                inputs = inputs.view(inputs.size(0), -1).float()
                if use_cuda:
                    inputs = inputs.cuda()
                inputs = Variable(inputs)
                z, outputs, mu, logvar = self.forward(inputs)

                loss = self.loss_function(outputs, inputs, z, mu, logvar)
                valid_loss += loss.data.item()*len(inputs)
                # total_loss += valid_recon_loss.data[0] * inputs.size()[0]
                # total_num += inputs.size()[0]
                gamma = self.get_gamma(z, mu, logvar).data.cpu().numpy()
                Y.append(labels.cpu().numpy())
                Y_pred.append(np.argmax(gamma, axis=1))

                # view reconstruct
                if visualize and batch_idx == 0:
                    n = min(inputs.size(0), 8)
                    comparison = torch.cat([inputs.view(-1, 1, 11, 20)[:n],
                                            outputs.view(-1, 1, 11, 20)[:n]])
                    save_image(comparison.data.cpu(),
                                 'results/vae/reconstruct/reconstruction_' + str(epoch) + '.png', nrow=n)

            Y = np.concatenate(Y)
            Y_pred = np.concatenate(Y_pred)
            #print("getting cluster acc")
            acc = cluster_acc(Y_pred, Y)
            # valid_loss = total_loss / total_num
            print("#Epoch %3d: lr: %.5f, Train Loss: %.5f, Valid Loss: %.5f, acc: %.5f" % (
                epoch, epoch_lr, train_loss / len(trainloader.dataset), valid_loss / len(validloader.dataset), acc[0]))

            # view sample
            if visualize:
                sample = Variable(torch.randn(LATENT_N, self.z_dim))
                if use_cuda:
                   sample = sample.cuda()
                sample = self.decode(sample).cpu()
                save_image(sample.data.view(LATENT_N, 11*20),
                           'results/vae/sample/sample_' + str(epoch) + '.png')

    def log_marginal_likelihood_estimate(self, x, num_samples):
        weight = torch.zeros(x.size(0))
        for i in range(num_samples):
            z, recon_x, mu, logvar = self.forward(x)
            zloglikelihood = log_likelihood_samples_unit_gaussian(z)
            dataloglikelihood = torch.sum(x*torch.log(torch.clamp(recon_x, min=1e-10))+
                (1-x)*torch.log(torch.clamp(1-recon_x, min=1e-10)), 1)
            log_qz = log_likelihood_samplesImean_sigma(z, mu, logvar)
            weight += torch.exp(dataloglikelihood + zloglikelihood - log_qz).data
        # pdb.set_trace()
        return torch.log(torch.clamp(weight/num_samples, min=1e-40))

In [ ]:
class CDRH3MotifDataset(Dataset):
    """
    pyTorch Dataloader conform class,
    requires implementation of __len__ and __getitem__
    """

    def __init__(self, csv_path: str, device: str = 'cuda:0'):
        """
        Args:
            csv_path (string): Path to the csv file with CDRH3 motifs.
            device (string): device where data is kept in memory
        """
        with open(csv_path, 'r') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)
            classes = set()
            data = list()
            for line in reader:
                data.append((line[0], line[1:]))
                classes |= set(line[0])

        encoding = {letter: np.eye(len(classes), dtype='float32')[i]
                    for i, letter in enumerate(classes)}
        label_encoding = {letter: i for i, letter in enumerate(classes)}
        dataset = np.zeros((len(data), len(data[0][0]), len(classes)),
                           dtype='int')
        labels = np.zeros((len(data), len(data[0][0])), dtype='int64')
        eval_data = dict()
        for i, line in enumerate(data):
            for jdx, letter in enumerate(line[0]):
                dataset[i, jdx, :] = encoding[letter]
                labels[i, jdx] = label_encoding[letter]
                eval_data[i] = line[1]

        # transform numpy array to torch tensor
        # and copy it on the correct device
        self.dataset = torch.from_numpy(dataset).to(device)
        self.labels = torch.from_numpy(labels).to(device)
        self.eval_data = eval_data
        self.encoding = encoding
        self.label_encoding = label_encoding
        self.reverse_encoding = {repr(i): j for j, i in encoding.items()}

    def __len__(self) -> int:
        return len(self.dataset)

    """
    __getitem__
    Output:
        (OneHotEncoding of data, IntegerEncoding of data,
        list with epitope, structure and antigen)
    """
    def __getitem__(self, idx: int) -> (FloatTensor):
        return (self.dataset[idx, :, :])

class CDRH3MotifDataset_labels(Dataset):
    """
    pyTorch Dataloader conform class,
    requires implementation of __len__ and __getitem__
    """

    def __init__(self, csv_path: str, device: str = 'cuda:0'):
        """
        Args:
            csv_path (string): Path to the csv file with CDRH3 motifs.
            device (string): device where data is kept in memory
        """
        with open(csv_path, 'r') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)
            classes = set()
            data = list()
            for line in reader:
                data.append((line[0], line[1:]))
                classes |= set(line[0])

        encoding = {letter: np.eye(len(classes), dtype='float32')[i]
                    for i, letter in enumerate(classes)}
        label_encoding = {letter: i for i, letter in enumerate(classes)}
        dataset = np.zeros((len(data), len(data[0][0]), len(classes)),
                           dtype='int')
        labels = np.zeros((len(data), len(data[0][0])), dtype='int64')
        eval_data = dict()
        for i, line in enumerate(data):
            for jdx, letter in enumerate(line[0]):
                dataset[i, jdx, :] = encoding[letter]
                labels[i, jdx] = label_encoding[letter]
                eval_data[i] = line[1]

        # transform numpy array to torch tensor
        # and copy it on the correct device
        self.dataset = torch.from_numpy(dataset).to(device)
        self.labels = torch.from_numpy(labels).to(device)
        self.eval_data = eval_data
        self.encoding = encoding
        self.label_encoding = label_encoding
        self.reverse_encoding = {repr(i): j for j, i in encoding.items()}
        with open(DATA_PATH, 'r') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)
            classes = set()
            data = list()
            for line in reader:
                data.append((line[0], line[1:]))
                classes |= set(line[0])
        labels = [x[1][2] for x in data]
        ag = labels
        ag_set = set()
        for element in ag:
            ag_set |= {element}
        ag_map = {name: i for i, name in enumerate(ag_set)}
        self.ag_color = torch.Tensor([ag_map[s] for s in ag]).to(device)
        print(self.ag_color.shape)
        self.dataset = self.dataset[1:10000,:,:]
        self.ag_color = self.ag_color[1:10000]
    def __len__(self) -> int:
        return len(self.dataset)

    """
    __getitem__
    Output:
        (OneHotEncoding of data, IntegerEncoding of data,
        list with epitope, structure and antigen)
    """
    def __getitem__(self, idx: int) -> (FloatTensor):
        return (self.dataset[idx, :, :],self.ag_color[idx])

In [ ]:
dataset = CDRH3MotifDataset(DATA_PATH, device=DEVICE)
dataset_val = CDRH3MotifDataset_labels(DATA_PATH, device = DEVICE)

In [ ]:
train_data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0,
                    drop_last = False)
val_data = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)

In [ ]:
!mkdir -p results/vae/reconstruct
!mkdir -p results/vae/sample

In [ ]:
vade = VaDE(input_dim=11*20, z_dim=LATENT_N, n_centroids=10, binary=True,
        encodeLayer=[200,200,400], decodeLayer=[400,200,200])

print("Initializing through GMM..")
vade.initialize_gmm(val_data)
vade.fit(train_data, val_data, lr=0.001, batch_size=BATCH_SIZE, num_epochs=EPOCHS, anneal=True,visualize = True)

In [ ]:
!ls results/vae/

In [ ]:
from IPython.display import Image
Image('results/vae/reconstruct/reconstruction_0.png',width=400,height=800)

In [ ]:
Image('results/vae/reconstruct/reconstruction_1.png',width=400,height=800)

In [ ]:
Image('results/vae/reconstruct/reconstruction_2.png',width=400,height=800)

In [ ]:
Image('results/vae/reconstruct/reconstruction_3.png',width=400,height=800)

In [ ]:
Image('results/vae/reconstruct/reconstruction_4.png',width=400,height=800)

In [ ]:
Image('results/vae/sample/sample_4.png',width=400,height=800)

In [ ]:
all_data = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,
                    drop_last = False)
all_z = np.zeros((len(dataset),LATENT_N),np.float32)
with torch.no_grad():
    for batch_idx, (inputs) in enumerate(all_data):
        inputs = inputs.view(inputs.size(0), -1).float()
        z = vade.forward(inputs)
        if z[0].shape[0] == BATCH_SIZE:
            all_z[batch_idx*BATCH_SIZE:batch_idx*BATCH_SIZE+BATCH_SIZE,:] = z[0].cpu().numpy()
        else:
            all_z[batch_idx*BATCH_SIZE:len(dataset),:] = z[0].cpu().numpy()

In [ ]:
do_eval(all_z,do_umap=False)    